## PI Members Management

In [1]:
import PySimpleGUI as sg
from backend_PI import * # Import tout ce qui est spécifique au projet

connect('PIPlanning')

backend_PI_mongo_model imported
/media/olivier/Donnees/Documents/Formations/Perso/PI_Planning/jupyter_files backend_PI_Utils imported
/media/olivier/Donnees/Documents/Formations/Perso/PI_Planning/jupyter_files global_variables imported
/media/olivier/Donnees/Documents/Formations/Perso/PI_Planning/jupyter_files backend_PI_Members imported
backend_PI_Projects imported
/media/olivier/Donnees/Documents/Formations/Perso/PI_Planning/jupyter_files backend_PI_Teams imported
/media/olivier/Donnees/Documents/Formations/Perso/PI_Planning/jupyter_files frontend_PI_Utils imported


MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

## create_member_gui

In [ ]:
def create_member_gui(info='Info'):
    sg.set_options(element_padding=(5, 10))

    projects_list=list_projects()
    comboproj = []
    for project in projects_list:
        comboproj.append(project.ProjectName)
    print(comboproj)
    
    team=[]
    combomembers=[]
    
    info_layout = [sg.T(info,font='Calibri 11',justification="left")]
 
    left_layout = [
        [sg.T('Project Selection', size=(20, 1),font='Calibri 11'), sg.Combo(comboproj,key='-PROJ-',enable_events=True,size=(20, 1),font='Calibri 11')],
        [sg.T('Team Name', size=(20, 1),key='-TXTTEAM-',font='Calibri 11',visible=False), sg.Combo(team,key='-TEAM-',enable_events=True,visible=False,size=(20, 1),font='Calibri 11')],
    ]
    
    bottom_layout=[[sg.T('Task Name',key='-TNAME-',size=(15,1),font='Calibri 11',visible=False),
                    sg.I("",key='-TNAMEI-',visible=False,size=(20,1))],
                  [sg.T('Task Member',key='-TMEMBER-',size=(15,1),font='Calibri 11',visible=False),
                   sg.Combo(combomembers,key='-TMEMBERI-',enable_events=True,size=(20, 1),font='Calibri 11',visible=False)]
                  ]
    
#    layout = [info_layout,[sg.Frame("Select perimeter", left_layout, vertical_alignment='top', pad=((10, 10), (10, 10)))],
#            [sg.B('Add', enable_events=True), sg.Cancel()]]
             
    layout = [info_layout,[sg.Frame("Select perimeter", left_layout, vertical_alignment='top', pad=((10, 10), (10, 10)))],
              [sg.Frame("Task", bottom_layout,key='-TASKS-', vertical_alignment='top',pad=((10, 10), (10, 10)),visible=False)],
              [sg.B('Add', enable_events=True), sg.Cancel()]]
        
    window = MyWindow('Create task', layout,finalize=True)
    window.my_move_to_center()
    
    while True:
        event, values = window.read()
#        print(event,values)
        
        if event == sg.WIN_CLOSED or event == 'Cancel':
#            print(event)
            window.close()
            break
 
        elif '-PROJ-' in event:
#           print(values['-PROJ-'])
            project=values['-PROJ-']
            list_teams = []
            teams_list=list_teams(project)
            for i in range(len(teams_list)):
#                print(teams_list[i][1])
                list_teams.append(teams_list[i][1])
            print(list_teams)
            window['-TXTTEAM-'].update(visible=True)
            window['-TEAM-'].update(values=list_teams,visible=True)
            

              
        elif '-TEAM-' in event:
            team=values['-TEAM-']
            print(team)

            tasks_lists=list_tasks(project,team,'All','All')
            
            combomembers=query_members_by_team(team)
            print(combomembers)
            
            titre='Lists of tasks for project: '+project+' and team:'+team
            window['-TASKS-'].update(value=titre,visible=True)
            window['-TNAME-'].update(visible=True)
            window['-TNAMEI-'].update(visible=True)
            window['-TMEMBER-'].update(visible=True)
            window['-TMEMBERI-'].update(values=combomembers,visible=True)
            
        elif event == 'Add':
            print(event,values)
            pass

## list_members_gui(teamid,page,linespage,info='info')
**WIP**

In [42]:
def list_members_gui(teamid,page,linespage,info='info'):
    if g.DEBUG_OL >= 1:
        print('--- function: list_members_gui(',teamid,page,linespage,info,')')
 
    #    global page
    members=[]
    members=query_members_by_team(teamid)
    items=len(members)
    if g.DEBUG_OL >= 1:
        print('items:',items)
        
    sg.set_options(element_padding=(5, 5))
#    list_teams=list_teams_all()
    layout = [[sg.T(info,font='Calibri 11',justification="left")],
              [sg.T('Member Name',font='Calibri 11', size=(20, 1)),
               sg.T('Member Firstname',font='Calibri 11',size=(20, 1)),
               sg.T('Member Alias',font='Calibri 11', size=(30, 1)),
               sg.T('Member Role',font='Calibri 11', size=(20, 1)),
               sg.T('Member Email',font='Calibri 11', size=(20, 1)),
              ]]
    idx=0
    for member in members:
        if g.DEBUG_OL >= 2:
            print('MemberID',member[0],'\tProjectID',member[6],'\tTeam:',member[7])

        row = [sg.I(member[1],disabled=True, font='Calibri 11', size=(20,1)),
               sg.I(member[3],disabled=True, font='Calibri 11',size=(30,1)),
               sg.I(member[2],disabled=True, font='Calibri 11',size=(20,1)),
               sg.I(member[9],disabled=True, font='Calibri 11',size=(20,1)),
               sg.I(member[4],disabled=True, font='Calibri 11',size=(20,1)),
              ]
        layout.append(row)
        idx+=1
   
    pagination = [[sg.B('<<', key='-BEGINING-'),
                   sg.B("<", key='-BACK-'),
                   sg.T(text=page, key='-PAGE-', size=(2, 1)),
                   sg.B(">", key='-NEXT-'),
                   sg.B(">>", key='-END-')
                   ]]
    layout += [[sg.Col(pagination, justification='right')]]
    layout += [[sg.B('Return')]]
               
    window = MyWindow('List of Members of team '+members[0][8], layout,keep_on_top=True, element_justification = 'center',finalize=True)
    window.my_move_to_center()
    
    while True:
        event1, values1 = window.read()
        if g.DEBUG_OL >= 2:
            print(event1,values1)
        if event1 == sg.WIN_CLOSED or event1 == 'Return':
            window.close()
            return(None)
            break
            
        elif event1 == ">":
            if teams.has_next:
                page += 1
                window.close()
                teams = list_members_page(page,team.ProjectID)
                list_all_teams_gui(page,teams,'ceci est l"info de base')
        elif event1 == "<":
            if teams.has_prev:
                page -= 1
                window.close()
                teams = list_members_page(page,team.ProjectID)
                list_all_teams_gui(page,teams,'ceci est l"info de base')
        elif event1 == "<<":
            if teams.has_prev:
                page = 1
                window.close()
                teams = list_members_page(page,team.ProjectID)
                list_all_teams_gui(page,teams,'ceci est l"info de base')
        elif event1 == ">>":
            if teams.has_next:
                page = teams.pages
                window.close()
                teams = list_members_page(page,team.ProjectID)
                list_all_teams_gui(page,teams,'ceci est l"info de base')


In [43]:
list_members_gui(1,1,5)

--- function: list_members_gui( 1 1 5 info )
--- function: query_members_by_team( 1 )
teamid: 1 Qtt members found: 3
[1]
[1, 2]
[1, 2, 5]
1
--- function: query_member_alias( 1 )
MemberID: 1
User Alias: oliboub
User Name: Boubert
User First Name: Olivier
User Email: oliboub@gmail.com
Project allocated: PIPlanning
Project Team allocated: PI
User Theme: LightBrown4
Team Role: Solution Owner
Member admin: False
Member First Connection: False
[1, 'Boubert', 'oliboub', 'Olivier', 'oliboub@gmail.com', 'LightBrown4', 'PIPlanning', 1, 'PI', 'Solution Owner', False, False]
2
--- function: query_member_alias( 2 )
MemberID: 2
User Alias: MacFly
User Name: TVV
User First Name: MC
User Email: mctvv@live.com
Project allocated: PIPlanning
Project Team allocated: PI
User Theme: LightBlue2
Team Role: Scrum Master
Member admin: False
Member First Connection: True
[2, 'TVV', 'MacFly', 'MC', 'mctvv@live.com', 'LightBlue2', 'PIPlanning', 1, 'PI', 'Scrum Master', False, True]
5
--- function: query_member_ali

In [27]:
print(__name__,'imported')

__main__ imported
